In [25]:
#Caricare le librerie
import tweepy
import pandas as pd
import numpy as np
import re
import json

In [26]:
consumer_key= "rrR0Znz98oKkjfB8b72oLH9Qc"
consumer_secret= "m7iOPX6iW8R3IalRf7x56YVeIb18o3I1bhUftg8JzlQ6fAjLgW"
access_token= "1442250607-3cJYKYyh7Ae19nMcv5YoxOIB2XlvgVX0ROaDSJx"
access_token_secret= "ZOnaaCTgO7DsKtmPjVruOlkvzBY8p6MbkKLjiYLdamBYE"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [27]:
#Scaricare in un dataframe i tweet in base ad un hashtag (until="2021-12-21")
#NB: hasattr(obj, name, /) Return whether the object has an attribute with the given name.
hashtag="#fedez"
list_tweet=[]
list_originals=[]
list_retweet=[]
n=100
for tweet in tweepy.Cursor(api.search_tweets,q=hashtag,count=100,lang="it").items(n):
  #print(tweet.text)
  #print('')
  try:
    list_originals.append(tweet.retweeted_status.user.screen_name) #creo lista dei tweetter originali
    list_retweet.append(json.dumps(tweet.retweeted_status._json)) #creo lista json dei retweet
  except:
    list_originals.append(None)
  list_tweet.append([tweet.created_at, tweet.id, tweet.text, 
                     tweet.favorite_count, tweet.retweet_count, 
                     tweet.user.location,tweet.user.screen_name,
                     hasattr(tweet, "retweeted_status"),
                     tweet.in_reply_to_screen_name])
tweets = pd.DataFrame(list_tweet[1:], 
                      columns=['date','id','text','like','n_rt','location','author',
                               "retweet","reply_name"])
tweets["retwitted_author"]=list_originals[1:]
tweets.head()

,date,id,text,like,n_rt,location,author,retweet,reply_name,retwitted_author
0,2021-12-19 10:37:01+00:00,1472516310282158088,RT @MikyS03: Oggi alle 14:00 su #canale5 14° e...,0,1,,carlo234556,True,None,MikyS03
1,2021-12-19 10:32:51+00:00,1472515264470298627,Solo mezza paginata per la scossa di terremoto...,0,0,,pazzoperrep,False,None,None
2,2021-12-19 10:32:37+00:00,1472515203153727495,Oggi alle 14:00 su #canale5 14° ed ultimo appu...,4,1,,MikyS03,False,None,None
3,2021-12-19 10:27:18+00:00,1472513865552482304,RT @Belzebu6663: Spaventati dal #Terremoto #Fe...,0,3,,LNN999,True,None,Belzebu6663
4,2021-12-19 09:53:13+00:00,1472505289333260292,RT @FabrizioDelpret: La cosa più interessante ...,0,1,,tabarro63,True,None,FabrizioDelpret


In [22]:
json.dumps?

Signature:
json.dumps(
    obj,
    *,
    skipkeys=False,
    ensure_ascii=True,
    check_circular=True,
    allow_nan=True,
    cls=None,
    indent=None,
    separators=None,
    default=None,
    sort_keys=False,
    **kw,
)
Docstring:
Serialize ``obj`` to a JSON formatted ``str``.

If ``skipkeys`` is true then ``dict`` keys that are not basic types
(``str``, ``int``, ``float``, ``bool``, ``None``) will be skipped
instead of raising a ``TypeError``.

If ``ensure_ascii`` is false, then the return value can contain non-ASCII
characters if they appear in strings contained in ``obj``. Otherwise, all
such characters are escaped in JSON strings.

If ``check_circular`` is false, then the circular reference check
for container types will be skipped and a circular reference will
result in an ``OverflowError`` (or worse).

If ``allow_nan`` is false, then it will be a ``ValueError`` to
serialize out of range ``float`` values (``nan``, ``inf``, ``-inf``) in
strict compliance of the JSON speci